In [ ]:
from detectdd import config
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow import keras

import root_config as rc
import pandas as pd
from detectdd.serializer import Serializer

from sklearn.model_selection import train_test_split

rc.configure()

bp_df = pd.read_csv(config.out_dir / 'vitals_data-10.csv')
print(bp_df.dtypes)

bp_df["charttime"] = bp_df["charttime"].astype("datetime64[s]")
bp_df["dose_b_time"] = bp_df["dose_b_time"].astype("datetime64[s]")

bp_df = bp_df.sort_values(by=["stay_id", "dose_b_time", "charttime"])

try:
    serializer = Serializer()
    cohort_with_icd = serializer.read_cohort()  # need to run 01-cohort.ipynb to produce the cohort
    print(len(cohort_with_icd))
    cohort_without_icd = serializer.read_cohort_with_no_icd()
    print(len(cohort_without_icd))
    cohort = pd.concat([cohort_with_icd, cohort_without_icd])
except FileNotFoundError:
    raise Exception("Need to run [01-cohort.ipynb] at least once to create the cohort file in the /out directory")

unique_subject_num_icd_codes = cohort[["subject_id", "num_icd_codes"]].drop_duplicates(subset="subject_id")
print(unique_subject_num_icd_codes)
print(bp_df)

bp_df = bp_df.drop_duplicates()

bp_df = pd.merge(left=bp_df, right=unique_subject_num_icd_codes, how="inner", on='subject_id')

bp_df["has_icd"] = ~bp_df['num_icd_codes'].isna()
bp_df["has_icd"].describe()
bp_df.head(500)

In [ ]:
bp_df

In [ ]:
bp_df[bp_df['stay_id'] == 30031264].sort_values(by='charttime')

In [ ]:
serializer = Serializer()
bp_df_no_icd = serializer.read_bp_results("10-no-icd")
bp_df_no_icd["has_icd"] = 0
bp_df_no_icd
df = bp_df
# df= pd.concat([bp_df, bp_df_no_icd])
# df = pd.DataFrame(df.head(5000)) #TODO remove me

In [ ]:
df["charttime"] = df["charttime"].astype("datetime64[s]")
df["dose_b_time"] = df["dose_b_time"].astype("datetime64[s]")

df.dtypes

In [ ]:

## convert chart_time to timestep relative to dose b time
df["timestep"] = df["charttime"] - df["dose_b_time"]
## and convert to float64 for tensor flow
df['timestep_float'] = df['timestep'].dt.total_seconds().astype('float64')

data_arrays = ["heart_rate", "sbp", "dbp", "mbp"]
for data in data_arrays:
    df[data] = df[data].astype('float64')

print(df.dtypes)

In [ ]:
df = df[["stay_id", "timestep", "timestep_float", "heart_rate", "sbp", "dbp", "mbp", "dose_b_time", "charttime",
         "has_icd"]].sort_values(["stay_id", "dose_b_time", "timestep"])
df.head(50)

# Interpolate
The data has missing values, use pandas built in interpolation to fill those based on the timestamp

In [ ]:
import pandas as pd

print(df.dtypes)
# df = df.reset_index()
df.set_index('charttime', inplace=True)
df.groupby(["stay_id", "has_icd", "dose_b_time"])


def interpolate_group(group):
    if group.name in data_arrays:
        # Perform interpolation for the group
        group = group.interpolate(method='time')

    return group


# Apply the interpolation function to each group
df = df.apply(interpolate_group).reset_index()


def list_of_floats(x):
    return x.astype(float).tolist()


group_df = df[["stay_id", "dose_b_time", "timestep_float", "heart_rate", "sbp", "dbp", "mbp", "has_icd"]]
print(df.dtypes)
grouped_df = group_df  

print(grouped_df.dtypes)

print(grouped_df.shape)
grouped_df.head(50)

# Remove outliers

In [ ]:
# reduce the size of the data arrays to the p999 to get rid of the extreme outliers
length = pd.DataFrame()
length["val"] = [len(array) for array in grouped_df.groupby(["stay_id", "dose_b_time"])["mbp"]]
print(length.describe(percentiles=[0.01, 0.025, 0.25, 0.5, 0.75, 0.975, 0.99, 0.999]))
print(grouped_df.dtypes)
max_size = 90  # p999

data_array_with_time = data_arrays.copy()
data_array_with_time.append("timestep_float")
print(data_array_with_time)
print(data_arrays)


def resize(s):
    s.head(max_size)


grouped_df = grouped_df.groupby(["stay_id", "dose_b_time"]).head(max_size)

max_sequence_length = max(len(array) for array in grouped_df.groupby(["stay_id", "dose_b_time"])["sbp"])
print(max_sequence_length)
print(grouped_df.dtypes)
grouped_df

# Unify timesteps
The time measurement data varies per dosage, this step aligns all the time steps to 10 minute intervals, and interpolates between our 
existing time measures to produce 12 hours of 10 minute timestaps 

In [ ]:
# first fill blank values through interpellation
from scipy.interpolate import interp1d


def interpolate_row(row):
    vals = pd.DataFrame()
    time_steps = np.array(row["timestep_float"])
    common_time_interval = range(0, 12 * 60, 10)

    for data_array in data_arrays:
        interp_func = interp1d(time_steps, row[data_array], kind='next', fill_value='extrapolate')
        vals["intr-" + data_array] = interp_func(common_time_interval)

    vals["common_timestep"] = common_time_interval

    return vals


# Apply the function to the specific subset of rows
interpolated_df = grouped_df.groupby(['stay_id', 'has_icd', 'dose_b_time']).apply(interpolate_row)
print(interpolated_df)
print(grouped_df.dtypes)
interpolated_df.head(50)

# Normalise and scale data

In [ ]:
from sklearn.preprocessing import RobustScaler
# 
features = ['intr-heart_rate', 'intr-sbp', 'intr-dbp', 'intr-mbp']
# Initialize the RobustScaler
scaler = RobustScaler()

# Fit and transform the column

for feature in features:
    interpolated_df[feature] = scaler.fit_transform(interpolated_df[[feature]])

print(interpolated_df)

In [ ]:
print(grouped_df.dtypes)

interpolated_df.groupby(['stay_id', 'dose_b_time']).count().describe()

In [ ]:
interpolated_df

In [ ]:
from numpy import shape
from keras_preprocessing.sequence import pad_sequences

target = 'has_icd'

# Extract features and timestep data

interpolated_df = interpolated_df.reset_index()
timesteps = interpolated_df.groupby(["stay_id", "dose_b_time"])['common_timestep'].count()

# Determine the maximum sequence length
max_sequence_length = max(timesteps)
print(f"Max sequence length {max_sequence_length}")

max_dosages = df.groupby("stay_id")["dose_b_time"].nunique().max()
print(f"Max dosages {max_dosages}")

# Organize data by stay_id
stay_ids = interpolated_df['stay_id'].unique()

X_sequences = []
y_sequences = []
zeros = np.zeros((max_sequence_length, len(features)))

cols = ['stay_id', 'dose_b_time'] + features
print(cols)
xs = interpolated_df[cols].values
shape(xs)

# Initialize an empty list to store arrays
result = []

# Iterate over unique values of key1
invalid_stays = []
for stay_id in stay_ids:
    # Filter the DataFrame for the current key1
    df_stays = interpolated_df[interpolated_df["stay_id"] == stay_id]

    # Create a list to store arrays for the current key1
    stays_array = []

    unique_doses = df_stays['dose_b_time'].unique()
    # Iterate over unique values of key2 within the current key1
    for dose_b_time in unique_doses:
    # Filter the DataFrame for the current key2
        df_dose = df_stays[df_stays["dose_b_time"] == dose_b_time]
    
        # Extract the "val" columns for the current key2
        val_columns = df_dose[features].values
        
        # Append the extracted values as an array to the list
        if ~np.any(np.isnan(val_columns)):
            stays_array.append(val_columns)
        else:
            invalid_stays.append(stay_id)
            print(f"Found array values with NaN {stay_id} - {val_columns}")
        
        
    # Pad out the second dimension with zeros
    while len(stays_array) < max_dosages:
        stays_array.append(zeros)
    
        # Convert the list of arrays to a NumPy array for the current key1
    stays_array = np.array(stays_array)
    y_sequences.append(df_stays[target].unique()[0])
    # Append the 3D array for the current key1 to the result list
    result.append(stays_array)
    

# Convert the result list to a NumPy array
X_sequences = np.array(result)

shape(X_sequences)

original_shape = X_sequences.shape
X_sequences = X_sequences.reshape(original_shape[0], original_shape[1] * original_shape[2], original_shape[3])

print(shape(X_sequences))
print(shape(y_sequences))

print(f"Found {len(invalid_stays)} - {invalid_stays}")


In [ ]:
y_sequences = np.array(y_sequences).astype(int)
print(shape(X_sequences))

X = X_sequences[:3000]
y = y_sequences[:3000]

print(shape(X))

print(shape(y))
test = pd.DataFrame(np.isnan(X.flatten()))

print(f"Found {len(invalid_stays)} - {invalid_stays}")
test[test[0]==True]

In [ ]:
grouped_df.head(2000).reset_index().groupby('stay_id')


In [ ]:
print(len(X))
print(len(y))

#Adjust the size of the testing set: we'll use 10% of the entire data. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

#Check the number of columns (features):
print(len(X_train))
print(shape(X_train))
print(shape(X_test))
print(shape(y_train))
print(shape(y_test))


In [ ]:
from keras.layers import LSTM, BatchNormalization, LayerNormalization, Dense

print(max_sequence_length)

model = keras.Sequential()

max_feature_dim = 4
LAYERS = [32,32,1]

model.add(keras.layers.Masking(mask_value=0))
model.add(LSTM(64))
model.add(keras.layers.Dense(1, activation='sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
classifier = model.fit(X_train, y_train, epochs=10, batch_size=200,
                       verbose=1)  #set verbose = 1 to see the fitting process

In [ ]:
# Plot Accuracy over the epochs
plt.plot(classifier.history['accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.title('model accuracy')
plt.legend(['train'], loc='upper left')
plt.show()

# Plot Loss over the epochs
plt.plot(classifier.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.title('model loss')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
# evaluate the keras model
loss, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))
print(round(loss,3),round(accuracy,3))


In [ ]:
print(round(loss,3),round(accuracy,3))

In [ ]:
from sklearn import metrics

# #Predict the testing set
predictions = (model.predict(X_test) > 0.5).astype(int)

#Accuracy classification score
acc = float(round(metrics.accuracy_score(y_test, predictions),3))

#Compute the balanced accuracy.
bacc = float(round(metrics.balanced_accuracy_score(y_test, predictions),3))

#Compute the Matthews correlation coefficient (MCC)
mcc = float(round(metrics.matthews_corrcoef(y_test, predictions),3))

#Compute the F1 score, also known as balanced F-score or F-measure.
f1 = float(round(metrics.f1_score(y_test, predictions),3))

#Show results as a DataFrame:
results = {'Accuracy' : [acc], 'Balanced Accuracy' : [bacc], 'MCC' : [mcc], 'F1-Score' : [f1]}
df_results = pd.DataFrame.from_dict(data = results, orient='columns')
print(df_results)